
# Trajectoires de tempêtes dans les réanalyses ERA5

Calepin à utiliser pour le tracé de trajectoires de tempêtes remarquables à partir des fichiers "tempête.txt" obtenus grâce au programme de tracking automatique du calepin "ERA5_storms_tracking_auto.ipynb".

In [ ]:
import os

import xarray as xr
import netCDF4

import numpy as np
from scipy.ndimage import gaussian_filter

from cartopy import config
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

from tqdm import tqdm

In [ ]:
storms=['Nov1982', 'Oct1987', 'Herta', 'Viviane', 'Lothar', 'Martin',
        'Klaus', 'Xynthia', 'Joachim', 'Zeus', 'Ophelia', 'Eleanor', 'Alex']

In [ ]:
projection=ccrs.NearsidePerspective(central_longitude=0.0, central_latitude=45.0)
bounds = [(-60, 35, 30, 70)]
coast = cfeature.NaturalEarthFeature(category='physical', scale='10m', facecolor='none', name='coastline')
                             
fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Major storms which affected France tracked in ERA5 reanalysis',loc='center',fontsize=14)
ax.add_feature(coast, edgecolor='gray')
ax.gridlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

for storm in storms:
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',skiprows=0, dtype='U13,float,float,int',unpack=True)
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    ax.plot(liste_lon,liste_lat, transform=ccrs.PlateCarree(), label=storm+' '+date1+'-'+date2)
    ax.text(liste_lon[0]+1, liste_lat[0], storm, verticalalignment='center', horizontalalignment='right',transform=ccrs.PlateCarree())

ax.legend(loc="upper center", ncol=3)
plt.show()

figname='./figs/tracks'
fig.savefig(figname+'.png')

In [ ]:
interval=int(input("Enter the desired time interval : "))

if not os.path.exists("./figs/tracking"):
    os.makedirs("./figs/tracking")

In [ ]:
for storm in storms:
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',skiprows=0,
                                                         dtype='U13,float,float,int',unpack=True)

    liste_time=liste_time[::interval]
    liste_lon=liste_lon[::interval]
    liste_lat=liste_lat[::interval]
    liste_pres=liste_pres[::interval]
    date1=str(liste_time[0])
    date2=str(liste_time[-1])

    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(storm,loc='center',fontsize=14)
    ax.plot(liste_time,liste_pres, color='red', label=storm)
    ax.scatter(liste_time,liste_pres, color='red')
    plt.grid()
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Mean sea level pressure (hPa)', fontsize=12)
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=90)
    ax.tick_params(axis='x', labelsize=10)
    plt.show()
    figname='./figs/tracking/'+storm+'_slp_evolution'
    fig.savefig(figname+'.png')

In [ ]:
def calculer_baisse_maximale_pression(pressions):
    pression_min = float('inf')
    pression_max = float('-inf')
    debut_baisse_max = 0
    fin_baisse_max = 0
    points_baisse_max = 0
    points_actuels = 0

    for i, pression in enumerate(pressions):
        if pression > pression_max:
            pression_max = pression
            points_actuels = 0
        if pression < pression_min:
            pression_min = pression
            points_actuels += 1
            if points_actuels > points_baisse_max:
                points_baisse_max = points_actuels
                debut_baisse_max = i - points_actuels + 1
                fin_baisse_max = i
        elif pression == pression_min:
            points_actuels += 1
        else:
            points_actuels = 0

    return pression_max - pression_min, points_baisse_max, debut_baisse_max, fin_baisse_max

def running_diff(arr, N):
    return np.array([arr[i] - arr[i-N] for i in range(N, len(arr))])

In [ ]:
for storm in storms:
    
    liste_time,liste_lon,liste_lat,liste_pres=np.loadtxt('./txt/'+storm+'.txt',skiprows=0,
                                                         dtype='U13,float,float,int',unpack=True)

    liste_time=liste_time[::interval]
    liste_lon=liste_lon[::interval]
    liste_lat=liste_lat[::interval]
    liste_pres=liste_pres[::interval]
    date1=str(liste_time[0])
    date2=str(liste_time[-1])
    
    # Maximum pressure fall
    baisse_maximale, points_baisse_max, debut_baisse_max, fin_baisse_max = calculer_baisse_maximale_pression(liste_pres)
    # 12h deepening rate
    N=int((12/interval))
    dr=running_diff(liste_pres, N)
    max_dr=np.min(dr)
    max_dr_ind=np.argmin(dr)
    print(storm)
    print("Maximum pressure fall (hPa) : ", baisse_maximale)
    print("Beginning of the deepening phase : ", liste_time[debut_baisse_max])
    print("End of the deepening phase : ", liste_time[fin_baisse_max])
    print("Number of elapsed hours during the maximum pressure fall : ", (points_baisse_max*interval)-interval)
    print("Corresponding mean pressure fall rate in hPa/h : ", baisse_maximale/(points_baisse_max*interval))
    print('Maximum instantaneus deepening rate (Bergeron) : ', -max_dr*np.sin(45*np.pi/180)/np.sin(liste_lat[max_dr_ind+int(N/2)]*np.pi/180)/12)
    print('Maximum instantaneus deepening rate date : ', liste_time[max_dr_ind+int(N/2)])

    fig = plt.figure(figsize=(15, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(storm,loc='center',fontsize=14)
    ax.plot(liste_time,liste_pres, color='red', label=storm)
    ax.scatter(liste_time,liste_pres, color='red')
    plt.grid()
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Mean sea level pressure (hPa)', fontsize=12)
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=90)
    ax.tick_params(axis='x', labelsize=10)
    plt.show()
    figname='./figs/tracking/'+storm+'_slp_evolution'
    fig.savefig(figname+'.png')